In [1]:
# Dependencies:
# pip: scikit-learn, anndata, scanpy
#
# Modified from the Python starter kit for the NeurIPS 2021 Single-Cell Competition.
# Parts with `TODO` are supposed to be changed by you.
#
# More documentation:
#
# https://viash.io/docs/creating_components/python/

In [2]:
import logging
import anndata as ad
import sys

from scipy.sparse import csc_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
import numpy as np

logging.basicConfig(level=logging.INFO)

In [3]:
## VIASH START
# Anything within this block will be removed by `viash` and will be
# replaced with the parameters as specified in your config.vsh.yaml.
meta = { 'resources_dir': '.' }

par = {
    'input_train_mod1': 'sample_data/openproblems_bmmc_multiome_starter/openproblems_bmmc_multiome_starter.train_mod1.h5ad',
    'input_train_mod2': 'sample_data/openproblems_bmmc_multiome_starter/openproblems_bmmc_multiome_starter.train_mod2.h5ad',
    'input_test_mod1': 'sample_data/openproblems_bmmc_multiome_starter/openproblems_bmmc_multiome_starter.test_mod1.h5ad',
    'distance_method': 'minkowski',
    'output': 'output.h5ad',
    'n_pcs': 50,
}
## VIASH END

In [4]:
method_id = 'basic_beans'
sys.path.append(meta['resources_dir'])

In [5]:
logging.info('Reading `h5ad` files...')
input_train_mod1 = ad.read_h5ad(par['input_train_mod1'])
input_train_mod2 = ad.read_h5ad(par['input_train_mod2'])
input_test_mod1 = ad.read_h5ad(par['input_test_mod1'])



INFO:root:Reading `h5ad` files...


In [6]:

# TODO: implement own method
from method import method
adata = method(input_train_mod1, input_train_mod2, input_test_mod1)

adata.uns["method_id"] = method_id



INFO:root:Performing dimensionality reduction on modality 1 values...
INFO:root:Performing dimensionality reduction on modality 2 values...
INFO:root:Running Linear regression...


In [7]:

from scipy.sparse import issparse
issparse(adata.X)

True

In [ ]:

logging.info('Storing annotated data...')
adata.write_h5ad(par['output'], compression = "gzip")


In [9]:
from pygam import LogisticGAM
from cellrank.ul.models import GAM as crGAM

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [10]:
gam = crGAM(input_train_mod1, input_train_mod2)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [19]:
from pygam import LinearGAM

In [28]:
from sklearn.decomposition import TruncatedSVD
logging.info('Performing dimensionality reduction on modality 1 values...')    
input_mod1 = ad.concat(
    {"train": input_train_mod1, "test": input_test_mod1},
    axis=0,
    join="outer",
    label="group",
    fill_value=0,
    index_unique="-"
)
    
embedder_mod1 = TruncatedSVD(n_components=50)
mod1_pca = embedder_mod1.fit_transform(input_mod1.X)

logging.info('Performing dimensionality reduction on modality 2 values...')
embedder_mod2 = TruncatedSVD(n_components=50)
mod2_pca = embedder_mod2.fit_transform(input_train_mod2.X)

# split dimred back up
X_train = mod1_pca[input_mod1.obs['group'] == 'train']
X_test = mod1_pca[input_mod1.obs['group'] == 'test']
y_train = mod2_pca
    
logging.info('Running Linear regression...')


INFO:root:Performing dimensionality reduction on modality 1 values...
INFO:root:Performing dimensionality reduction on modality 2 values...
INFO:root:Running Linear regression...


In [35]:
len(X_train[0])

50

In [37]:
mod2_pca.shape

(221, 50)

In [40]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X_train, mod2_pca)

KNeighborsRegressor(n_neighbors=2)

In [42]:
y_pred = neigh.predict(X_test)

In [39]:



# Train the model on the PCA reduced modality 1 and 2 data
gam = LinearGAM().fit(X_train, mod2_pca)


ValueError: y data is not in domain of logit link function. Expected domain: [0.0, 1.0], but found [-3.33, 14.27]

In [ ]:
y_pred = reg.predict(X_test)


In [44]:
y_pred = neigh.predict(X_test)

In [43]:

# Project the predictions back to the modality 2 feature space
y_pred = y_pred @ embedder_mod2.components_

# Store as sparse matrix to be efficient. Note that this might require
# different classifiers/embedders before-hand. Not every class is able
# to support such data structures.
y_pred = csc_matrix(y_pred)

adata = ad.AnnData(
    X=y_pred,
    obs=input_test_mod1.obs,
    var=input_train_mod2.var,
    uns={
        'dataset_id': input_train_mod1.uns['dataset_id'],
        'method_id': 'starter_kit'
    },
)
